In [ ]:
import pickle
with open('/workspace/persistent/ct_df_posts_single_preds.pkl', 'rb') as file:
    # Load the contents of the PKL file
    ndf = pickle.load(file)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
#ndf = pd.read_csv('ndf.csv')
#ndf=ndf.drop_duplicates(subset=['shortcode'])
#ndf=ndf.dropna(subset=['Category'])
#ndf=ndf.loc[ndf['Followees']>0]
train_split, test_split = train_test_split(ndf, test_size=0.5, random_state=42)
test_split, eval_split = train_test_split(test_split, test_size=0.5, random_state=42)

In [ ]:
import pickle
with open('/workspace/persistent/train_split_thales.pkl', 'wb') as file:
    pickle.dump(train_split, file)
import pickle
with open('/workspace/persistent/test_split_thales.pkl', 'wb') as file:
    pickle.dump(test_split, file)
import pickle
with open('/workspace/persistent/eval_split_thales.pkl', 'wb') as file:
    pickle.dump(eval_split, file)

In [ ]:
!pip install emoji
!pip install transformers
!pip install datasets
!pip install evaluate

In [ ]:
datasets_dict = {"train_split": train_split, "eval_split": eval_split,  "test_splits": test_split, 'ndf':ndf}

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
import datasets
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", force_download=False)

def tokenize_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors='pt')

def transform_dataset(df):
    df["has_disclosures"] = df["has_disclosures"].astype(bool)
    df["caption"] = df["caption"].astype(str)
    if isinstance(df, pd.DataFrame):
        return datasets.Dataset.from_pandas(df[["caption", "has_disclosures"]].rename(columns={"caption": "text", "has_disclosures": "label"}))
    else:
        return [transform_dataset(k) for k in df]
  
def tokenize_dataset(df):
      if isinstance(df, list):
        return [tokenize_dataset(k) for k in df]
      else:
        return df.map(tokenize_function, batched=True)
    
hf_datasets = {k: transform_dataset(v) for k, v in datasets_dict.items()}
tokenized_datasets = {k: tokenize_dataset(v) for k, v in hf_datasets.items()}

# hf_train = Dataset.from_pandas(task_train[["caption_task", "has_disclosures", "country", "size", "date"]].rename(columns={"caption_task": "text", "has_disclosures": "label"}))
# hf_test = Dataset.from_pandas(task_test[["caption_task", "has_disclosures", "country", "size", "date"]].rename(columns={"caption_task": "text", "has_disclosures": "label"}))
# hf_gold = Dataset.from_pandas(gold_data[["caption_task", "label", "country", "size", "year"]].rename(columns={"caption_task": "text"}))

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
import numpy as np
import evaluate

acc = evaluate.load("accuracy", average="macro")
p = evaluate.load("precision", average="macro")
r = evaluate.load("recall", average="macro")
f1 = evaluate.load("f1", average="macro")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = acc.compute(predictions=predictions, references=labels)["accuracy"]
    precision = p.compute(predictions=predictions, references=labels)["precision"]
    recall = r.compute(predictions=predictions, references=labels)["recall"]
    f1m = f1.compute(predictions=predictions, references=labels)["f1"]
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1m}

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2, force_download=True,).to("cuda")
training_args = TrainingArguments(num_train_epochs=3, output_dir="test_trainer", evaluation_strategy="epoch", logging_strategy="epoch",)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train_split"],
    eval_dataset=tokenized_datasets["eval_split"],
    compute_metrics=compute_metrics
)
trainer.train()
trainer.evaluate()

In [ ]:
model.save_pretrained('/workspace/persistent/model')
tokenizer.save_pretrained('/workspace/persistent/tokenizer')

In [ ]:
from scipy.special import softmax

def predict(text, model, tokenizer):
    encoded_input = tokenizer(text, return_tensors="pt", truncation=True, max_length=tokenizer.model_max_length).to("cuda")
    output = model(**encoded_input, output_hidden_states=True)
    scores = output[0][0].detach().cpu().numpy()
    scores = softmax(scores)
    #return labels[scores.argmax()]
    return output

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoConfig
from transformers import AutoModelForSequenceClassification
import torch
tokenizer = AutoTokenizer.from_pretrained('/workspace/persistent/ttokenizer')
config = AutoConfig.from_pretrained("/workspace/persistent/tmodel/config.json")
model = AutoModelForSequenceClassification.from_config(config)
model.load_state_dict(torch.load('/workspace/persistent/tmodel/pytorch_model.bin'))
model.to('cuda')  # Move the model to CUDA if available

In [ ]:
test_split['id']=0
test_split['id']=range(len(test_split))
test_split = test_split.set_index(test_split['id'])

In [ ]:
train_split['id']=0
train_split['id']=range(len(train_split))
train_split = train_split.set_index(train_split['id'])

In [ ]:
test_split=test_split.dropna(subset=['caption'])

In [ ]:
train_split=train_split.dropna(subset=['caption'])

In [ ]:
from tqdm import tqdm
text_state=[]
for i in tqdm(range(len(test_split)), desc="Processing"):
    with torch.no_grad():
        state=(predict(test_split['caption'][i], model=model, tokenizer=tokenizer)[1][-1][-1][-1].to('cuda')).to('cpu')
    text_state.append(state)
text_state[0]

In [ ]:
from tqdm import tqdm
text_state_train=[]
for i in tqdm(range(len(train_split)), desc="Processing"):
    with torch.no_grad():
        state=(predict(train_split['caption'][i], model=model, tokenizer=tokenizer)[1][-1][-1][-1].to('cuda')).to('cpu')
    text_state_train.append(state)
text_state_train[0]

In [ ]:
import pickle
with open('/workspace/persistent/text_state_thales.pkl', 'wb') as file:
    pickle.dump(text_state, file)

In [ ]:
import pickle
with open('/workspace/persistent/text_state_thales_train.pkl', 'wb') as file:
    pickle.dump(text_state_train, file)

In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm

# Define the self-attention module
class SelfAttention(nn.Module):
    def __init__(self, embedding_dim, num_heads):
        super(SelfAttention, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=num_heads)

    def forward(self, x):
        # Apply self-attention
        output, _ = self.attention(x, x, x)

        return output.squeeze()

# Define the parameters
embedding_dim = 768
num_heads = 4

# Create an instance of the self-attention module
self_attention = SelfAttention(embedding_dim, num_heads).cuda()

# Convert the given tensor to a PyTorch tensor and move to GPU

# Apply self-attention to all elements in the tensor
text_attention = []
with tqdm(total=len(test_split), desc="Applying Attention") as progress_bar:
    for i,rows in test_split.iterrows():
        # Apply self-attention
        output = (text_state[i].unsqueeze(0).to('cuda'))
        with torch.no_grad():
            output = output.cpu()
        text_attention.append(output)
        progress_bar.update(1)


In [ ]:
import torch
import torch.nn as nn
from tqdm import tqdm

# Define the self-attention module
class SelfAttention(nn.Module):
    def __init__(self, embedding_dim, num_heads):
        super(SelfAttention, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=num_heads)

    def forward(self, x):
        # Apply self-attention
        output, _ = self.attention(x, x, x)

        return output.squeeze()

# Define the parameters
embedding_dim = 768
num_heads = 4

# Create an instance of the self-attention module
self_attention = SelfAttention(embedding_dim, num_heads).cuda()

# Convert the given tensor to a PyTorch tensor and move to GPU

# Apply self-attention to all elements in the tensor
text_attention_train = []
with tqdm(total=len(train_split), desc="Applying Attention") as progress_bar:
    for i,rows in train_split.iterrows():
        # Apply self-attention
        output = (text_state_train[i].unsqueeze(0).to('cuda'))
        with torch.no_grad():
            output = output.cpu()
        text_attention_train.append(output)
        progress_bar.update(1)


In [ ]:
import pickle
with open('/workspace/persistent/text_attention_thales.pkl', 'wb') as file:
    pickle.dump(text_attention, file)

In [ ]:
import pickle
with open('/workspace/persistent/text_attention_thales_train.pkl', 'wb') as file:
    pickle.dump(text_attention_train, file)

In [ ]:
text_representation = []
with tqdm(total=len(test_split), desc="Calculating the Image Score") as progress_bar:
    for i,rows in test_split.iterrows():
        xi=text_state[i].unsqueeze(0).to('cuda')*text_attention[i].unsqueeze(0).to('cuda')
        with torch.no_grad():
            xi = xi.cpu()
        text_representation.append(xi)
        progress_bar.update(1)

In [ ]:
text_representation_train = []
with tqdm(total=len(train_split), desc="Calculating the Image Score") as progress_bar:
    for i,rows in train_split.iterrows():
        xi=text_state_train[i].unsqueeze(0).to('cuda')*text_attention_train[i].unsqueeze(0).to('cuda')
        with torch.no_grad():
            xi = xi.cpu()
        text_representation_train.append(xi)
        progress_bar.update(1)

In [ ]:
import pickle
with open('/workspace/persistent/text_representation_thales.pkl', 'wb') as file:
    pickle.dump(text_representation, file)

In [ ]:
import pickle
with open('/workspace/persistent/text_representation_thales_train.pkl', 'wb') as file:
    pickle.dump(text_representation_train, file)